### 딥러닝 실습 과제 1주차 - 데이터 전처리

다음 세 가지 활동을 해봅시다.

01. **이미지 & 레이블 로드**: JSON 파일과 이미지 데이터를 PyTorch Dataset 형식으로 변환
02. **이미지 전처리**: 크기 조정, 정규화
03. **학습/검증/테스트 데이터 분할**


TTTDataset.zip을 불러와 문제에서 요하는 코드를 구현하세요.

💡 **데이터 구조**  
- **`image_black`** : 이미지 데이터  
- **`labels`** : 타겟 데이터  

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import json

import glob
import os

#### 00. 클래스
정의한 클래스를 이용해 실행해 주세요.

In [7]:
class TTTDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        """
        틱택토 데이터셋을 PyTorch Dataset 형태로 변환.
        :param image_paths: 이미지 파일 경로 리스트
        :param label_paths: 레이블 JSON 파일 경로 리스트
        :param transform: 이미지 전처리 변환
        """
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform
        self.data = self._load_data()


    def _load_data(self):
        """ 이미지 & 레이블 로드 """
        data = []
        for img_path, lbl_path in zip(self.image_paths, self.label_paths):
            # 이미지를 흑백(Grayscale)로 변환
            image = Image.open(img_path).convert("L")  # "RGB" 대신 "L" 사용

            # JSON 레이블 로드
            with open(lbl_path, 'r') as f:
                labels = json.load(f)

            # 레이블을 숫자로 변환 (O=1, X=-1, blank=0)
            label_tensor = torch.tensor(
                [1 if v == "O" else -1 if v == "X" else 0 for v in labels.values()],
                dtype=torch.float32
            )
            data.append((image, label_tensor))

        return data


    def __len__(self):
        """ 데이터셋 크기 반환 """
        return len(self.data)


    def __getitem__(self, idx):
        """ 데이터셋에서 idx 번째 샘플(이미지 & 레이블)을 가져오는 역할 """
        image, label = self.data[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

#### 01. 이미지 & 레이블 로드: JSON 파일과 이미지 데이터를 PyTorch Dataset 형식으로 변환

In [8]:
# 1. 이미지와 레이블 파일 경로 로드
image_dir = os.path.join("TTTDataset", "image_black")
labels_dir = os.path.join("TTTDataset", "labels")

# 모든 이미지와 라벨 파일의 경로 가져오기
image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")))
label_paths = sorted(glob.glob(os.path.join(labels_dir, "*.json")))

print(f"이미지 파일 수: {len(image_paths)}")
print(f"라벨 파일 수: {len(label_paths)}")
print(f"첫 번째 이미지 경로: {image_paths[0]}")
print(f"첫 번째 레이블 경로: {label_paths[0]}")

이미지 파일 수: 365
라벨 파일 수: 453
첫 번째 이미지 경로: TTTDataset/image_black/01.jpg
첫 번째 레이블 경로: TTTDataset/labels/01_labels.json


#### 02. 이미지 전처리: 크기 조정, 정규화

In [ ]:
# 2. 이미지 전처리를 위한 transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),               # 이미지 크기 조정
    transforms.ToTensor(),                       # PIL 이미지를 텐서로 변환 (0-255 → 0-1)
    transforms.Normalize(mean=[0.5], std=[0.5])  # 흑백 이미지 정규화 (-1 ~ 1)
])

# 전처리가 잘 되는지 확인하기 - 첫 번째 이미지에 적용
sample_image = Image.open(image_paths[0]).convert("L")
processed_image = transform(sample_image)
print(f"원본 이미지 크기: {sample_image.size}")
print(f"전처리 후 텐서 크기: {processed_image.shape}")
print(f"텐서 값 범위: {processed_image.min():.4f} ~ {processed_image.max():.4f}")

원본 이미지 크기: (512, 512)
전처리 후 텐서 크기: torch.Size([1, 224, 224])
텐서 값 범위: -1.0000 ~ 1.0000


#### 03. 학습 / 검증 / 테스트 데이터 분할

In [11]:
from torch.utils.data import random_split

# 전체 데이터셋 생성
full_dataset = TTTDataset(image_paths, label_paths, transform=transform)

# 데이터 분할 비율 설정
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# 전체 데이터 개수
total_size = len(full_dataset)
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)
test_size = total_size - train_size - val_size

# 데이터셋 분할
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, 
    [train_size, val_size, test_size]
)

# DataLoader 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 분할된 데이터셋 정보 출력
print(f"전체 데이터셋 크기: {total_size}")
print(f"학습 데이터셋 크기: {len(train_dataset)}")
print(f"검증 데이터셋 크기: {len(val_dataset)}")
print(f"테스트 데이터셋 크기: {len(test_dataset)}")

# 배치 크기 확인
for images, labels in train_loader:
    print(f"배치 형태: {images.shape}, {labels.shape}")
    break

전체 데이터셋 크기: 365
학습 데이터셋 크기: 255
검증 데이터셋 크기: 54
테스트 데이터셋 크기: 56
배치 형태: torch.Size([32, 1, 224, 224]), torch.Size([32, 9])
